In [3]:
import pandas as pd

data_113 = pd.read_csv(r"C:\資料探勘\原始資料\DM113.csv")

In [1]:
import pandas as pd
data_113 = pd.read_excel(r"C:\資料探勘\經緯度資料標記\113_unlabel.xlsx") 


In [3]:
label_data_113 = pd.read_excel(r"C:\資料探勘\經緯度資料標記\113_label.xlsx")

In [4]:
nomissing_label_data_113 = label_data_113[- label_data_113[['經度','緯度']].isna().any(axis=1)]
missing_label_data_113 = label_data_113[label_data_113[['經度','緯度']].isna().any(axis=1)]



In [ ]:
missing_label_data_113

In [4]:
def processing_address(row):
    
    if "臺北市" not in row["土地位置建物門牌"]:  
        
        return "臺北市" + row["鄉鎮市區"] + row["土地位置建物門牌"] 
    
    else:
        
        return  row["土地位置建物門牌"]

    

In [5]:
data_113 = data_113.copy()
data_113['土地位置建物門牌'] =  data_113[["鄉鎮市區","土地位置建物門牌"]].apply(processing_address,axis=1)

remove the another purpose for land 

In [6]:
import numpy as np



def delete_interupt_address(string):
    
    try:
    
        if string[-2:] == "地號":

            return "unnormal"

        else :

            return "normal"
        
    
    except Exception as e:
        
        return "unnormal"

    
    


In [7]:
import numpy as np
data_113["土地位置建物門牌"]= data_113["土地位置建物門牌"].apply(lambda x:x if delete_interupt_address(x) == "normal" else np.nan)

In [8]:
data_113= data_113.dropna(subset=["土地位置建物門牌"])


In [ ]:
%pip install selenium

In [ ]:
import time
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



options = Options()
options.chrome_executable_path = r"C:\資料探勘\chromedriver.exe"
driver = webdriver.Chrome(options=options)
driver.get("https://map.tgos.tw/TGOSCloudMap")


close_button = WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "bootbox-close-button.close.btn-close"))
        )
close_button.click()

address_List = []

n = 0
for idx,address in enumerate(missing_label_data_113["土地位置建物門牌"].to_list()): 
    
    try: 
    
        # Find and clear the search input field (timeout: 2 seconds)
        search = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "form-control.border-end-0.border-start-0"))
        )

        search.clear()

        # Input the address
        search.send_keys(address)

        # Wait for the search button to be clickable and click it (timeout: 2 seconds)
        search_button = WebDriverWait(driver, 60).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "fa-solid.fa-magnifying-glass.search"))
        )

        search_button.click()

        time.sleep(random.uniform(5, 10))

        # Wait for the <li> element to be present (timeout: 2 seconds)
        li_element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="map2DArea"]/div[4]/div/div/div[2]/ul/li')))


        # Extract latitude and longitude
        latitude = li_element.get_attribute("data-lat")
        longitude = li_element.get_attribute("data-lng")
        

        
        # Store the latitude and longitude in the corresponding row of the DataFrame
        # Store in the specific row using the index
        
        
        address_List.append([idx,latitude,longitude])
        
        missing_label_data_113.at[idx,'經度'] = latitude
        missing_label_data_113.at[idx,'緯度'] = longitude
        
        
        n += 1
        print(f"Processed address {address}: Latitude = {latitude}, Longitude = {longitude},the number of data {n} ")
        
    except Exception as e:
        
        
        try:
                
            print(f"Timeout waiting for result for address {address}. Skipping...")

            missing_label_data_113.at[idx, '經度'] = pd.NA
            missing_label_data_113.at[idx, '緯度'] = pd.NA

            address_List.append([idx, None, None])

            time.sleep(120) 
            close_button = WebDriverWait(driver, 300).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "btn.btn-danger.bootbox-accept"))
            )
            close_button.click()
            time.sleep(10)
            
            continue
            
        
        except Exception as e:
            
            print("can't find any element in the search. Skipping ...")
            
            missing_label_data_113.at[idx, '經度'] = pd.NA
            missing_label_data_113.at[idx, '緯度'] = pd.NA
            
            
            search = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "form-control.border-end-0.border-start-0"))
            )

            search.clear()
            
            continue
            
   
             
            

C:\Users\ruizhong\AppData\Local\Temp\ipykernel_15900\2129225499.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_label_data_113.at[idx,'經度'] = latitude
C:\Users\ruizhong\AppData\Local\Temp\ipykernel_15900\2129225499.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '25.091031' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  missing_label_data_113.at[idx,'經度'] = latitude
C:\Users\ruizhong\AppData\Local\Temp\ipykernel_15900\2129225499.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '121.521547' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

Processed address 臺北市士林區基河路２３８號七樓之１０: Latitude = 25.091031, Longitude = 121.521547,the number of data 1 
Processed address 臺北市士林區忠誠路一段１７３巷２４號二樓: Latitude = 25.108219, Longitude = 121.531891,the number of data 2 
Processed address 臺北市士林區天母北路３６巷８號四樓: Latitude = 25.12037, Longitude = 121.525926,the number of data 3 
Processed address 臺北市士林區大南路３３５號五樓: Latitude = 25.088197, Longitude = 121.516261,the number of data 4 
Processed address 臺北市士林區承德路四段３巷２９號二樓之２: Latitude = 25.078707, Longitude = 121.522372,the number of data 5 
Processed address 臺北市大同區哈密街５９巷２弄９之２號五樓: Latitude = 25.078707, Longitude = 121.522372,the number of data 6 
Processed address 臺北市大同區大龍街９１巷１２號四樓之１: Latitude = 25.078707, Longitude = 121.522372,the number of data 7 
Processed address 臺北市大同區承德路一段２４巷１號九樓: Latitude = 25.078707, Longitude = 121.522372,the number of data 8 
Processed address 臺北市大安區延吉街１３１巷１弄１２號五樓: Latitude = 25.078707, Longitude = 121.522372,the number of data 9 
Processed address 臺北市大安區忠孝東路四段１５號十一樓: Latitude = 25

In [ ]:
missing_label_data_113

In [12]:
pd.set_option("display.max_row",None)
label_data_113 = pd.concat([nomissing_label_data_113,missing_label_data_113],axis=0)



In [13]:
label_data_113.to_excel(r'C:\資料探勘\經緯度資料標記\113_label.xlsx')


PermissionError: [Errno 13] Permission denied: 'C:\\資料探勘\\經緯度資料標記\\113_label.xlsx'

In [11]:
data_113= pd.DataFrame(data_113)

In [12]:
data_113.to_excel(r"C:\資料探勘\經緯度資料標記\113_unlabel.xlsx")

In [12]:
pd.DataFrame({"地址":extract_lat_lon_112["土地位置建物門牌"]}).to_csv("C:\\資料探勘\\資料探勘作業\\address.csv")